In [36]:
import os
from dotenv import load_dotenv

load_dotenv()

pine_key_api_key = os.getenv("PINE_CONE_API_KEY")

In [18]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

index_name = "hybrid-search-langchain-pinecone"

# initialize pinecone client
pc = Pinecone(api_key=pine_key_api_key)

#create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, #dimension of dense vector (hugging face)
        metric='dotproduct', #for sparse values
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )     
    )
   


In [19]:
index = pc.Index(index_name)
index

In [ ]:
# create dense vector
from langchain_community.embeddings import HuggingFaceEmbeddings

os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")
embedding = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"  
)

embedding

/var/folders/mt/_ncgm8vx00b50yjykvqyd7b80000gn/T/ipykernel_8041/1687944941.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [31]:
# creating sparse vector
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder.default()
bm25_encoder

In [32]:
corpus = ["The quick brown fox jumps over the lazy dog",
        "The lazy dog is brown",
        "The fox is brown"]

bm25_encoder.fit(corpus)

bm25_encoder.dump("bm25_encoder.json")

bm25_encoder = BM25Encoder().load("bm25_encoder.json")


100%|██████████| 3/3 [00:00<00:00, 1533.94it/s]


In [33]:
retriever = PineconeHybridSearchRetriever(embeddings=embedding, sparse_encoder=bm25_encoder, index=index)   

In [ ]:
retriever

In [ ]:
retriever.add_texts(["The quick brown fox jumps over the lazy dog",
        "The lazy dog is brown",
        "The fox is brown"])

In [ ]:
response = retriever.invoke("Which fox is brown?")
response